<a href="https://colab.research.google.com/github/EmmanuelADAM/IntelligenceArtificiellePython/blob/master/SolutionDetectionAlertesTensorBoard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Détection d'alertes par réseaux de neurones

Dans cet exercice, le but est de créer un système très basique de détection de messages douteux  par Réseaux de Neurones en se basant uniquement sur des mots clés trouvés dans les sujets de mails.

Une version plus évoluée étudierait aussi le texte transformé en sac de mots.


---
*L’observation du contenu des communications par les gouvernements dans un but de sécurité permettrait de détecter les mails potentiellement dangereux et de lever des alertes.
Cependant la masse d’information échangée est trop importante pour garantir une bonne analyse.
Des réseaux de neurones peuvent être utilisés pour guider la détection de problèmes potentiels.*

*Il s’agit donc ici de définir un réseau de neurones et de lui faire apprendre à détecter des textes à risque.*

---

Les exemples d’apprentissage seront constitués :
- d’une série de valeurs indiquant si oui ou non un terme est présent dans le texte analysé,
- de la réponse attendue (dans la ligne danger)

Pour chaque texte, des mots-clés sont détectés, et le danger potentiel du texte est indiqué. Voici les exemples d’apprentissage :
 - 'bombe', 'feu' => danger
 - 'bombe', 'patisserie' => _
 - 'bombe', 'acide' => danger
 - 'bombe', 'canon' => danger
 - 'fille', 'canon' => _
 - 'mec', 'canon' => _
 - 'patisserie', 'acide' => _
 - 'canon', 'balle' => danger
 - 'balle', 'tennis' => _
 - 'fille', 'acide' => danger
 
 Il faut transformer cela en un ensembles de vecteurs d'entrées et de sorties.

| bombe | feu | patisserie | fille | canon | mec | acide | balle | tennis | *danger* |
|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
| 1., | 1., | 0., | 0., | 0., | 0., | 0., | 0., | 0. |  1  |
| 1., | 0., | 0., | 0., | 0., | 0., | 1., | 0., | 0. |  1  |
| 1., | 0., | 0., | 0., | 1., | 0., | 0., | 0., | 0. |  1  |
| 0., | 0., | 0., | 0., | 1., | 0., | 0., | 1., | 0. |  1  |
| 0., | 0., | 0., | 1., | 0., | 0., | 1., | 0., | 0. |  1  |
| 1., | 0., | 1., | 0., | 0., | 0., | 0., | 0., | 0. |  0  |
| 0., | 0., | 0., | 1., | 1., | 0., | 0., | 0., | 0. |  0  |
| 0., | 0., | 0., | 0., | 1., | 1., | 0., | 0., | 0. |  0  |
| 0., | 0., | 1., | 0., | 0., | 0., | 1., | 0., | 0. |  0  |
| 0., | 0., | 0., | 0., | 0., | 0., | 0., | 1., | 1. |  0  |



---
## Importer les librairies

In [1]:
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np
#pour les graphiques
import matplotlib.pyplot as plt
#SI bug plus loin lors du dessin des graphiques, ajouter ces 2 lignes
#import os
#os.environ['KMP_DUPLICATE_LIB_OK']='True'


---
#### installation du tableau de bord tensorflow pour étudier l'apprentissage
Enregistre des données sur l'évolution des performances dans un fichier log qui sera visualisé par des outils graphiques

In [2]:
##Chargement du tableau de bord tensorflow
%load_ext tensorboard

In [3]:
#besoin du package sur les dates
import datetime

In [4]:
# Efface tous les anciens logs (attention au repertoire)
%rm -rf ./alertesLogs/

---

### Définir les entrées et sorties attendues

Les entrées correspondent à la présence des mots clés par dans les textes.
Ici, les entrées d'apprentissage sont consituées de 70% de l'exemple présenté (donc 7 lignes de  9 valeurs 0,1). Les sorties d'apprentissage correspondent au fait que chaque entrée soit un message dangereux ou non.

Les entrées et sorties de validations sont donc constitués des 30% restants


In [5]:
#transformer les données en entrées et sorties
# attention, il vaut mieux rester entre 0 & 1, surtout pour la sortie !

tableau = np.array([
[1.,1.,0.,0.,0.,0.,0.,0.,0.,1], 
[1.,0.,0.,0.,0.,0.,1.,0.,0.,1], 
[1.,0.,0.,0.,1.,0.,0.,0.,0.,1], 
[0.,0.,0.,0.,1.,0.,0.,1.,0.,1], 
[0.,0.,0.,1.,0.,0.,1.,0.,0.,1], 
[1.,0.,1.,0.,0.,0.,0.,0.,0.,0], 
[0.,0.,0.,1.,1.,0.,0.,0.,0.,0], 
[0.,0.,0.,0.,1.,1.,0.,0.,0.,0], 
[0.,0.,1.,0.,0.,0.,1.,0.,0.,0], 
[0.,0.,0.,0.,0.,0.,0.,1.,1.,0]], float)

keywords = ("bombe", "feu", "patisserie", "fille", "canon", "mec", "acide", "balle", "tennis")    
nb_lignes = tableau.shape[0]
nb_val = len(keywords)


In [6]:
#on prend un pourcentage de lignes pour apprendre, le reste pour valider
#on mélange les lignes pour pouvoir avoir des exemples d'apprentissage
# et de validation homogènes (au cas où les exemples auraient été saisis dans un ordre précis)
# mais le mieux est de mieux choisir ces exemples qui ont un impact important dans la qualité de l'apprentissage
np.random.shuffle(tableau)

#nb de lignes d'apprentissage
nb_lignes_app = nb_lignes * 70 // 100
#nb de lignes de validation
nb_lignes_val = nb_lignes - nb_lignes_app

# les premieres colonnes sont les entrees 
# la derniere colonne est la sortie

#on remplit les entrees et sorties d'apprentissage
entrees_app = tableau[0:nb_lignes_app, 0:nb_val]
sorties_app = tableau[0:nb_lignes_app, nb_val:nb_val+1]

#on remplit les entrees et sorties de validation
entrees_val = tableau[nb_lignes_app:nb_lignes, 0:nb_val]
sorties_val = tableau[nb_lignes_app:nb_lignes, nb_val:nb_val+1]

In [7]:
sorties_val

array([[1.],
       [0.],
       [0.]])

---

### Choisir le modèle de réseau
- ici les couches sont séquentielles

In [8]:
model = Sequential()

2022-04-25 11:39:35.520444: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


---

### Définir l'architecture du réseau
Choisissez la struture du réseau, le nb de couches cachées, etc.
- une première couche composée de 
  - autant de neurones en entrée que de mots clés, plus le neurone BIAS (ou non)
  - x neurones en sortie 
- une dernière couche composée de
  - y neurones en entrée (ceux de la couche précédente) et 
  - de 1 neurone en sortie (Spam ou non)


In [9]:
#une premiere couche constituée d'autant de neurones en entrée que de mots clés, 6 en sortie...
model.add(Dense(4, use_bias=True, input_shape=(nb_val,), activation='tanh'))

#une couche intermédiaire, 3 neurones en sortie et forcement 6 en entrees...
model.add(Dense(4, use_bias=True, activation='sigmoid'))

#une couche intermédiaire, 3 neurones en sortie et forcement 6 en entrees...
model.add(Dense(4, use_bias=True, activation='sigmoid'))

#une derniere couche constituée de 1 neurone en sortie, 
# nb neurones de la couche précédente en entrée et activation sigmoide
model.add(Dense(1, activation='sigmoid'))

---

### Compiler le  réseau
- ici, on précise que l'algo de correction d'erreur est *'adam'*, et que l'erreur calculée est la moyenne des valeurs absolues des erreurs commises. On indique également que l'on veut voir apparaître en plus la précision de l'apprentissage (accuracy).

(vous pouvez chosir un autre optimizer et un autre calcul de loss)

In [10]:
model.compile(optimizer='adamax', loss='MSE', metrics=['accuracy'])

----
#### <font color="red">Ajout de la volonté d'enregistrer dans un log les etapes d'apprentissage</code>


In [11]:
log_dir = "alertesLogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

2022-04-25 11:39:35.586931: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-04-25 11:39:35.586952: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2022-04-25 11:39:35.587001: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.


---

### Entraîner le réseau 
- ici on  le fait  'parler' (verbose=2) si le nb de cycles d'apprentissage (epochs) est court
- <font color="red">*Et on indique l'enregistreur de logs (callbacs)*</code>


In [12]:
tests = model.fit(entrees_app, sorties_app, 
                    validation_data=(entrees_val, sorties_val),
                    epochs=500, verbose=0,
                    callbacks=[tensorboard_callback])

2022-04-25 11:39:35.634467: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-04-25 11:39:36.470557: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-04-25 11:39:36.470574: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2022-04-25 11:39:36.472670: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2022-04-25 11:39:36.474355: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2022-04-25 11:39:36.476778: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: alertesLogs/fit/20220425-113935/train/plugins/profile/2022_04_25_11_39_36
2022-04-25 11:39:36.477556: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to alertesLogs/fit/20220425-113935/train/plugins/profile/2022_04_25_11_39_36/Ma

---

## Utilisation du tableau de bord


In [13]:
%tensorboard --logdir alertesLogs/fit

---
## Utilisation du réseau
Quelle est la probabilité que les textes contenant les mots :
1. bombe et feu
2. mec patisserie feu

soient des textes suspects ?

In [14]:
def build_input(mots):
    inputs = np.zeros(len(keywords))
    i = 0
    for mot in mots:
        inputs[tuple.index(keywords, mot)] = 1
    return inputs

input_test = build_input(( 'feu', 'bombe'))
entreesTests = np.array([input_test], float)

predictions = model.predict(entreesTests)
print(predictions)

input_test = build_input(('mec', 'patisserie', 'feu'))
entreesTests = np.array([input_test], float)

predictions = model.predict(entreesTests)
print(predictions)

[[0.655429]]
[[0.38988662]]
